### Other Temp

In [115]:
import unicodedata
import datetime as dt
import os
import sys
import re
import numpy as np
import pandas as pd
import math
from pathlib import Path
import nltk
from nltk import word_tokenize 
import shutil
from os.path import isfile, join
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import locale
from locale import atof
from geopy.geocoders import Nominatim
from geopy.point import Point
from geopy.exc import GeocoderTimedOut
from functools import reduce

##### Functions

In [116]:
# custom character alphabet for word
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß]+", re.IGNORECASE|re.UNICODE) # German
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓعٮێڔ۶ؚۧۜڤۏإٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ؍ي؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨٹڵڪظٶۭ١ڭەڨحٕ؎ٺڷٰ۪۫ڻڥۛڑڟټآڡغګ؊ّٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفرڇچڝ۴بۈٽڕۡھةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻشڍ؇۵كخ\ا]+", re.IGNORECASE|re.UNICODE) # Arabic

In [117]:
def convert_one_character_letter(text):
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. \n
    convert_one_character_letter(text): text is any string word or sentence.
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    return new_text

In [118]:
def clean_text(text, custom_alp=tr): # for only string in specific language
    '''This function extract custom_alp character from string \n
    clean_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.
    '''
    # text_clean = re.findall(custom_alp, text)
    text_result_list = []
    text_list = re.findall(r'\S+', text)
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_result = " ".join(text_result_list)
    return text_result

In [119]:
def lower_func(text):
    '''This function convert string character to lowercase. \n
    lower_func(text=str): text is any string word or sentence
    '''
    string_lower = str.lower(text)
    return string_lower

In [120]:
def upper_func(text):
    '''This function convert string character to uppercase. \n
    upper_func(text=str): text is any string word or sentence
    '''
    string_upper = str.upper(text)
    return string_upper

In [121]:
def capitalize_func(text):
    '''This function convert string character to capitalize case. \n
    capitalize_func(text=str): text is any string word or sentence
    '''
    string_capitalize = str.capitalize(text)
    return string_capitalize

In [122]:
def title_func(text):
    '''This function convert string character to title as each word start with capital letter. \n
    title_func(text=str): text is any string word or sentence
    '''
    string_title = str.title(text)
    return string_title

In [123]:
def character_clean_lower_text(text, custom_alp=tr): # for only string in specific language
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. And also extract custom_alp character from string and convert to lowercase. \n
    character_clean_lower_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.  
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    text_result_list = []
    text_list = re.findall(r'\S+', new_text)  # like as word tokenize
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_join = " ".join(text_result_list)
    text_result = str.lower(text_join)
    return text_result

In [124]:
def convert_to_float(value):
    '''This function convert string numeric value to float type. Like as 1.200,15
    convert_to_float(value): value is a string numeric value
    '''
    value = value.replace(".","").replace(",",".")
    return float(value)

In [125]:
def whitespace_del(text):
    '''This function provides removing left right white space.\n
    whitespace_del(text): text is a string value as word or sentence.
    '''
    text_var = str(text)
    text_var = text_var.strip()  # rstrip lstrip
    return text_var

In [126]:
def del_first_word(text):
    '''This function delete sentence first word \n
    del_first_word(text): text is a sentence.  
    '''
    # Metni kelimelere ayır
    words = text.split()

    # İlk kelimeyi sil
    words.pop(0)

    # Kelimeleri tekrar birleştir
    new_text = " ".join(words)
    return new_text

#### HB Temp

In [ ]:
df_ham = pd.read_excel(r"C:\Users\user\Desktop\HB HAM.xlsx")
df_ham

,UniqueIdentifier,Buybox Sırası,Satıcı Stok Kodu,SKU
0,NaN,3,ENT-HB604808211318,HBCV00001FAQ4X
1,NaN,3,ENT-HB604808211271,HBCV00001FAQ2Y
2,NaN,1,ENT-HB604808211387,HBCV00001FDICE
3,NaN,1,ENT-HB604808211950,HBCV00001V218U
4,NaN,1,ENT-HB604808211967,HBCV00001V218N
...,...,...,...,...
1306,NaN,1,ENTV-HB47320017854,HBCV00006ZCBGQ
1307,NaN,1,ENT-HB8699243467674504-1,HBCV00006ZCCFG
1308,NaN,1,ENT-HB0037905486,HBCV00006ZCBUJ
1309,NaN,1,ENTV-HB32478610045,HBCV00006ZCEOT


In [ ]:
df_ham["SKU"] = df_ham["SKU"].apply(lambda code: whitespace_del(code))
df_ham

,UniqueIdentifier,Buybox Sırası,Satıcı Stok Kodu,SKU
0,NaN,3,ENT-HB604808211318,HBCV00001FAQ4X
1,NaN,3,ENT-HB604808211271,HBCV00001FAQ2Y
2,NaN,1,ENT-HB604808211387,HBCV00001FDICE
3,NaN,1,ENT-HB604808211950,HBCV00001V218U
4,NaN,1,ENT-HB604808211967,HBCV00001V218N
...,...,...,...,...
1306,NaN,1,ENTV-HB47320017854,HBCV00006ZCBGQ
1307,NaN,1,ENT-HB8699243467674504-1,HBCV00006ZCCFG
1308,NaN,1,ENT-HB0037905486,HBCV00006ZCBUJ
1309,NaN,1,ENTV-HB32478610045,HBCV00006ZCEOT


In [ ]:
df_all = pd.read_excel(r"C:\Users\user\Desktop\HB SON.xlsx")
df_all

,SKU,Barkod
0,OUTVARIKY68672,0078257686728
1,HBV00000TY39B,2496695636224
2,sr45,241067121317799
3,68672-00,ERS78257686728
4,AX20349,3699948455151
...,...,...
2842,000063,739339869971
2843,000062,739339869957
2844,000112,8697671470960
2845,000133,8809213765551


In [ ]:
df_all["SKU"] = df_all["SKU"].apply(lambda code: whitespace_del(code))
df_all

,SKU,Barkod
0,OUTVARIKY68672,0078257686728
1,HBV00000TY39B,2496695636224
2,sr45,241067121317799
3,68672-00,ERS78257686728
4,AX20349,3699948455151
...,...,...
2842,000063,739339869971
2843,000062,739339869957
2844,000112,8697671470960
2845,000133,8809213765551


In [ ]:
df_output = pd.merge(df_ham, df_all, how="left", on="SKU")
df_output.drop_duplicates(inplace=True)
df_output

,UniqueIdentifier,Buybox Sırası,Satıcı Stok Kodu,SKU,Barkod
0,NaN,3,ENT-HB604808211318,HBCV00001FAQ4X,NaN
1,NaN,3,ENT-HB604808211271,HBCV00001FAQ2Y,NaN
2,NaN,1,ENT-HB604808211387,HBCV00001FDICE,NaN
3,NaN,1,ENT-HB604808211950,HBCV00001V218U,NaN
4,NaN,1,ENT-HB604808211967,HBCV00001V218N,NaN
...,...,...,...,...,...
1307,NaN,1,ENTV-HB47320017854,HBCV00006ZCBGQ,NaN
1308,NaN,1,ENT-HB8699243467674504-1,HBCV00006ZCCFG,NaN
1309,NaN,1,ENT-HB0037905486,HBCV00006ZCBUJ,NaN
1310,NaN,1,ENTV-HB32478610045,HBCV00006ZCEOT,NaN


In [ ]:
df_output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1312 entries, 0 to 1311
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   UniqueIdentifier  0 non-null      float64
 1   Buybox Sırası     1312 non-null   int64  
 2   Satıcı Stok Kodu  1312 non-null   object 
 3   SKU               1312 non-null   object 
 4   Barkod            33 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 51.4+ KB


In [ ]:
df_output = df_output.dropna(subset=["SKU", "Barkod"])
df_output

,UniqueIdentifier,Buybox Sırası,Satıcı Stok Kodu,SKU,Barkod
425,NaN,1,HBCV00000RQ3BN,HBCV00000RQ3BN,8696472600958
478,NaN,1,HBCV00003FRPQ2,HBCV00003FRPQ2,00000TYV-1405
479,NaN,2,HBCV00003EOZYW,HBCV00003EOZYW,İNCİTANEMc170029b-2a1
549,NaN,1,HBCV00002MRKPW,HBCV00002MRKPW,stilinburada768fa977-25d
653,NaN,2,HBCV00003B71V9,HBCV00003B71V9,1116320157863
682,NaN,2,HBCV00000EDDWH,HBCV00000EDDWH,8682315112993-1
683,NaN,2,HBCV00000EDDWH,HBCV00000EDDWH,8682315112993
684,NaN,2,HBCV00004EEMZZ,HBCV00004EEMZZ,8682315114577
685,NaN,2,HBCV000025TYN8,HBCV000025TYN8,8682315114195
687,NaN,1,HBCV000025TYNA,HBCV000025TYNA,8682315114096


In [ ]:
# df_output.to_excel(r"C:\Users\user\Desktop\HB Eşleşen.xlsx", index=False)

In [ ]:
df_all2 = pd.read_excel(r"C:\Users\user\Desktop\HB LOCAL.xlsx")
df_all2.drop_duplicates(inplace=True)
df_all2

,SKU,Barkod
0,HBCV00002T3FGP,604808214197
1,HBCV00004BLZM7,K8683372407008
2,HBCV00004BLZEX,S8683372407008
3,HBCV00004BLZEW,Y8683372407008
4,HBCV000003F3NY,K8683372407008
...,...,...
417,HBCV00005DJ8XU,vps100321657468
418,HBCV000020G4H7,vps100321657468
419,HBCV00001X9ZS15,vps100321657468
420,HBCV00001MWJLC,0042624576


In [ ]:
df_all2["SKU"] = df_all2["SKU"].apply(lambda code: whitespace_del(code))
df_all2

,SKU,Barkod
0,HBCV00002T3FGP,604808214197
1,HBCV00004BLZM7,K8683372407008
2,HBCV00004BLZEX,S8683372407008
3,HBCV00004BLZEW,Y8683372407008
4,HBCV000003F3NY,K8683372407008
...,...,...
417,HBCV00005DJ8XU,vps100321657468
418,HBCV000020G4H7,vps100321657468
419,HBCV00001X9ZS15,vps100321657468
420,HBCV00001MWJLC,0042624576


In [ ]:
df_output2 = pd.merge(df_ham, df_all2, how="left", on="SKU")
df_output2.drop_duplicates(inplace=True)
df_output2

,UniqueIdentifier,Buybox Sırası,Satıcı Stok Kodu,SKU,Barkod
0,NaN,3,ENT-HB604808211318,HBCV00001FAQ4X,NaN
1,NaN,3,ENT-HB604808211271,HBCV00001FAQ2Y,NaN
2,NaN,1,ENT-HB604808211387,HBCV00001FDICE,8690741756005
3,NaN,1,ENT-HB604808211387,HBCV00001FDICE,NRG413285161-1
4,NaN,1,ENT-HB604808211387,HBCV00001FDICE,ÇG782302202356
...,...,...,...,...,...
1401,NaN,1,ENTV-HB47320017854,HBCV00006ZCBGQ,NaN
1402,NaN,1,ENT-HB8699243467674504-1,HBCV00006ZCCFG,NaN
1403,NaN,1,ENT-HB0037905486,HBCV00006ZCBUJ,NaN
1404,NaN,1,ENTV-HB32478610045,HBCV00006ZCEOT,NaN


In [ ]:
df_output2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1406 entries, 0 to 1405
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   UniqueIdentifier  0 non-null      float64
 1   Buybox Sırası     1406 non-null   int64  
 2   Satıcı Stok Kodu  1406 non-null   object 
 3   SKU               1406 non-null   object 
 4   Barkod            249 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 55.1+ KB


In [ ]:
df_output2 = df_output2.dropna(subset=["SKU", "Barkod"])
df_output2

,UniqueIdentifier,Buybox Sırası,Satıcı Stok Kodu,SKU,Barkod
2,NaN,1,ENT-HB604808211387,HBCV00001FDICE,8690741756005
3,NaN,1,ENT-HB604808211387,HBCV00001FDICE,NRG413285161-1
4,NaN,1,ENT-HB604808211387,HBCV00001FDICE,ÇG782302202356
5,NaN,1,ENT-HB604808211387,HBCV00001FDICE,8684076000144
6,NaN,1,ENT-HB604808211387,HBCV00001FDICE,8684076000304
...,...,...,...,...,...
1076,NaN,1,ENT-HB0043551371454,HBCV00006FF9QO,0043551371454
1077,NaN,1,ENT-HB0040115765646,HBCV00006FFH29,0040115765646
1080,NaN,1,ENT-HB00401286467,HBCV00006FFHKK,00401286467
1082,NaN,1,ENT-HB00401347643,HBCV00006FFHKM,8698898312897


In [ ]:
# df_output2.to_excel(r"C:\Users\user\Desktop\HB Eşleşen2.xlsx", index=False)

In [ ]:
set1 = set(df_output["SKU"])
set2 = set(df_output2["SKU"])

In [ ]:
set1.intersection(set2)

{'HBCV00000AISQZ',
 'HBCV00000EDDWH',
 'HBCV00001MWJLC',
 'HBCV00001P4CHD',
 'HBCV000023CQYY',
 'HBCV00002MRKPW',
 'HBCV00003EOZYW',
 'HBCV00003FRPQ2',
 'HBCV00006091EF'}

In [ ]:
df_output_all = pd.concat([df_output, df_output2], axis=0)
df_output_all.drop_duplicates(subset=["SKU","Barkod"], inplace=True)
df_output_all.reset_index(drop=True, inplace=True)
df_output_all["SKU"] = df_output_all["SKU"].apply(lambda code: whitespace_del(code))
df_output_all

,UniqueIdentifier,Buybox Sırası,Satıcı Stok Kodu,SKU,Barkod
0,NaN,1,HBCV00000RQ3BN,HBCV00000RQ3BN,8696472600958
1,NaN,1,HBCV00003FRPQ2,HBCV00003FRPQ2,00000TYV-1405
2,NaN,2,HBCV00003EOZYW,HBCV00003EOZYW,İNCİTANEMc170029b-2a1
3,NaN,1,HBCV00002MRKPW,HBCV00002MRKPW,stilinburada768fa977-25d
4,NaN,2,HBCV00003B71V9,HBCV00003B71V9,1116320157863
...,...,...,...,...,...
277,NaN,1,ENT-HB0043551371454,HBCV00006FF9QO,0043551371454
278,NaN,1,ENT-HB0040115765646,HBCV00006FFH29,0040115765646
279,NaN,1,ENT-HB00401286467,HBCV00006FFHKK,00401286467
280,NaN,1,ENT-HB00401347643,HBCV00006FFHKM,8698898312897


In [ ]:
df_output_all["SKU"].nunique()

177

In [ ]:
df_output_all.to_excel(r"C:\Users\user\Desktop\HB Eşleşenler.xlsx", index=False)

In [ ]:
sku_list = list(set(df_output_all["SKU"]))
sku_list

['HBCV00005RNUZY',
 'HBCV00006AD0GU',
 'HBCV00004L1YQX',
 'HBCV00005NTBUJ',
 'HBCV000058GVXV',
 'HBCV00006083LT',
 'HBCV000050B25J',
 'HBCV00006FFH29',
 'HBCV00005C3NNX',
 'HBCV00006091EF',
 'HBCV00005DSZW4',
 'HBCV00004EPXDE',
 'HBCV00005VQQC7',
 'HBCV00005BZ40P',
 'HBCV00004B9V3K',
 'HBCV000052Q2U8',
 'HBCV00004T2GF3',
 'HBCV00005V20HK',
 'HBCV000001X1U9',
 'HBCV00005L5V78',
 'HBCV00004ELB1P',
 'HBCV00004DNGS2',
 'HBCV00004J9W60',
 'HBCV00005S71B9',
 'HBCV00002MRKPW',
 'OUTVARIKY68672',
 'HBCV000050A3V5',
 'HBCV00003S170B',
 'HBCV000050PUCV',
 'HBCV00006FF9QO',
 'HBCV00005DT2LG',
 'HBCV000025TYNA',
 'HBCV000069RSZN',
 'HBCV00005D749Q',
 'HBCV00005V2D5K',
 'HBCV00003B71V9',
 'HBCV00005CLX4G',
 'HBCV00004BNTCX',
 'HBCV00006FFHKM',
 'HBCV00005VTIVW',
 'HBCV00004KI8QG',
 'HBCV00004EDSHS',
 'HBCV00005O2033',
 'HBCV000060E8XJ',
 'HBCV00001MWJLC',
 'HBCV000067PK5I',
 'HBCV00005B86XE',
 'HBCV000040EDXA',
 'HBCV00005ROF5N',
 'HBCV000064FWWA',
 'HBCV000066TNB6',
 'HBCV000052Q3MJ',
 'HBCV00004B

In [ ]:
df_ham.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   UniqueIdentifier  0 non-null      float64
 1   Buybox Sırası     1311 non-null   int64  
 2   Satıcı Stok Kodu  1311 non-null   object 
 3   SKU               1311 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 41.1+ KB


In [ ]:
df_no_sku = df_ham[~df_ham["SKU"].isin(sku_list)]
df_no_sku

,UniqueIdentifier,Buybox Sırası,Satıcı Stok Kodu,SKU
0,NaN,3,ENT-HB604808211318,HBCV00001FAQ4X
1,NaN,3,ENT-HB604808211271,HBCV00001FAQ2Y
3,NaN,1,ENT-HB604808211950,HBCV00001V218U
4,NaN,1,ENT-HB604808211967,HBCV00001V218N
5,NaN,1,ENT-HB604808211981,HBCV00001V9NWP
...,...,...,...,...
1306,NaN,1,ENTV-HB47320017854,HBCV00006ZCBGQ
1307,NaN,1,ENT-HB8699243467674504-1,HBCV00006ZCCFG
1308,NaN,1,ENT-HB0037905486,HBCV00006ZCBUJ
1309,NaN,1,ENTV-HB32478610045,HBCV00006ZCEOT


In [ ]:
df_no_sku.to_excel(r"C:\Users\user\Desktop\HB Ham Edit.xlsx", index=False)

In [ ]:
df_ent_all = pd.read_excel(r"C:\Users\user\Desktop\Entegra All Sales.xlsx")
df_ent_all

,id,order_number,datetime,entegration,status,status_name,company,cargo_code,invoice_country,invoice_city,...,cargo_fee_type_name,cargo_packet_type,cargo_packet_quantity,grand_total_desi,model,product_name,store_product_name,invoice_name,pov_productCode,total_product_quantity
0,201036,2383110310,2024-09-05 10:54:27,trendyol,2,Onaylandı,ali özdemir,7330017026985454,NaN,İstanbul,...,Satıcı Öder,Dosya,1,0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...","6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sakl...","6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",NaN,1
1,201035,2383102583,2024-09-05 10:50:42,trendyol,2,Onaylandı,Tuğba Sayar,7330017026917196,NaN,İstanbul,...,Satıcı Öder,Dosya,1,0,004382,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2 Adet Çok Amaçlı Multi İnce Temizlik Fırçası ...,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,NaN,7
2,201033,2383091113,2024-09-05 10:43:15,trendyol,2,Onaylandı,Tuba Koca,7330017026782381,NaN,Ankara,...,Satıcı Öder,Dosya,1,0,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...","Ahşap Özel Isim Baskılı, Oval Kedi Köpek Mama ...","Ahşap Özel Isim Baskılı, Oval Kedi Köpek Mama ...",NaN,1
3,201025,2383085464,2024-09-05 10:40:26,trendyol,2,Onaylandı,Doğa Özkan,7330017026738593,NaN,İstanbul,...,Satıcı Öder,Dosya,1,0,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...","Ahşap Özel Isim Baskılı, Oval Kedi Köpek Mama ...","Ahşap Özel Isim Baskılı, Oval Kedi Köpek Mama ...",NaN,2
4,201027,2383076067,2024-09-05 10:34:55,trendyol,2,Onaylandı,Damla Beşer,7330017026623479,NaN,Sivas,...,Satıcı Öder,Dosya,1,0,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...","Portatif Katlanabilir Masa, Piknik Masası, Kam...","Portatif Katlanabilir Masa, Piknik Masası, Kam...",220647971025,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188034,83601,207423367318-1,2024-01-01 00:18:00,N11,1,Yeni Siparis,MELTEM KOCA,112054836588352,NaN,Kütahya,...,Satıcı Öder,Dosya,1,0,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",Portatif Katlanabilir Piknik Masası + Çaydanlı...,Portatif Katlanabilir Piknik Masası + Çaydanlı...,160620230178,1
188035,83598,1935764998,2024-01-01 00:14:43,trendyol,2,Onaylandı,Sıla Dikel,7330012502018770,NaN,Adıyaman,...,Satıcı Öder,Dosya,1,0,002979,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,3'lü Çok Amaçlı Kutu Mega Çizgili Kahverengi 6...,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,NaN,3
188036,83596,1935755636,2024-01-01 00:02:03,trendyol,2,Onaylandı,özlem yıldız,7330012501918644,NaN,Aydın,...,Satıcı Öder,Dosya,1,0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...","6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sakl...","6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",NaN,1
188037,83597,201757367316-1,2024-01-01 00:02:00,N11,1,Yeni Siparis,ekrem çelik,112085256588357,NaN,Kocaeli,...,Satıcı Öder,Dosya,1,0,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",Portatif Katlanabilir Piknik Masası + Çaydanlı...,Portatif Katlanabilir Piknik Masası + Çaydanlı...,160620230178,1


In [ ]:
df_ent_all = df_ent_all.drop_duplicates(subset="id")
df_ent_all

,id,order_number,datetime,entegration,status,status_name,company,cargo_code,invoice_country,invoice_city,...,cargo_fee_type_name,cargo_packet_type,cargo_packet_quantity,grand_total_desi,model,product_name,store_product_name,invoice_name,pov_productCode,total_product_quantity
0,201036,2383110310,2024-09-05 10:54:27,trendyol,2,Onaylandı,ali özdemir,7330017026985454,NaN,İstanbul,...,Satıcı Öder,Dosya,1,0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...","6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sakl...","6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",NaN,1
1,201035,2383102583,2024-09-05 10:50:42,trendyol,2,Onaylandı,Tuğba Sayar,7330017026917196,NaN,İstanbul,...,Satıcı Öder,Dosya,1,0,004382,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2 Adet Çok Amaçlı Multi İnce Temizlik Fırçası ...,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,NaN,7
2,201033,2383091113,2024-09-05 10:43:15,trendyol,2,Onaylandı,Tuba Koca,7330017026782381,NaN,Ankara,...,Satıcı Öder,Dosya,1,0,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...","Ahşap Özel Isim Baskılı, Oval Kedi Köpek Mama ...","Ahşap Özel Isim Baskılı, Oval Kedi Köpek Mama ...",NaN,1
3,201025,2383085464,2024-09-05 10:40:26,trendyol,2,Onaylandı,Doğa Özkan,7330017026738593,NaN,İstanbul,...,Satıcı Öder,Dosya,1,0,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...","Ahşap Özel Isim Baskılı, Oval Kedi Köpek Mama ...","Ahşap Özel Isim Baskılı, Oval Kedi Köpek Mama ...",NaN,2
4,201027,2383076067,2024-09-05 10:34:55,trendyol,2,Onaylandı,Damla Beşer,7330017026623479,NaN,Sivas,...,Satıcı Öder,Dosya,1,0,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...","Portatif Katlanabilir Masa, Piknik Masası, Kam...","Portatif Katlanabilir Masa, Piknik Masası, Kam...",220647971025,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188034,83601,207423367318-1,2024-01-01 00:18:00,N11,1,Yeni Siparis,MELTEM KOCA,112054836588352,NaN,Kütahya,...,Satıcı Öder,Dosya,1,0,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",Portatif Katlanabilir Piknik Masası + Çaydanlı...,Portatif Katlanabilir Piknik Masası + Çaydanlı...,160620230178,1
188035,83598,1935764998,2024-01-01 00:14:43,trendyol,2,Onaylandı,Sıla Dikel,7330012502018770,NaN,Adıyaman,...,Satıcı Öder,Dosya,1,0,002979,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,3'lü Çok Amaçlı Kutu Mega Çizgili Kahverengi 6...,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,NaN,3
188036,83596,1935755636,2024-01-01 00:02:03,trendyol,2,Onaylandı,özlem yıldız,7330012501918644,NaN,Aydın,...,Satıcı Öder,Dosya,1,0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...","6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sakl...","6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",NaN,1
188037,83597,201757367316-1,2024-01-01 00:02:00,N11,1,Yeni Siparis,ekrem çelik,112085256588357,NaN,Kocaeli,...,Satıcı Öder,Dosya,1,0,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",Portatif Katlanabilir Piknik Masası + Çaydanlı...,Portatif Katlanabilir Piknik Masası + Çaydanlı...,160620230178,1


In [ ]:
df_ent_all.to_excel(r"C:\Users\user\Desktop\Entegra Sales All.xlsx", index=False)

In [ ]:
df_ent_all_select = df_ent_all[["store_product_name", "model","pov_productCode","entegration"]]
df_ent_all_select

,store_product_name,model,pov_productCode,entegration
0,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sakl...",003362,NaN,trendyol
1,2 Adet Çok Amaçlı Multi İnce Temizlik Fırçası ...,004382,NaN,trendyol
2,"Ahşap Özel Isim Baskılı, Oval Kedi Köpek Mama ...",001602,NaN,trendyol
3,"Ahşap Özel Isim Baskılı, Oval Kedi Köpek Mama ...",001602,NaN,trendyol
4,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",002583,220647971025,trendyol
...,...,...,...,...
188034,Portatif Katlanabilir Piknik Masası + Çaydanlı...,002583,160620230178,N11
188035,3'lü Çok Amaçlı Kutu Mega Çizgili Kahverengi 6...,002979,NaN,trendyol
188036,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sakl...",003362,NaN,trendyol
188037,Portatif Katlanabilir Piknik Masası + Çaydanlı...,002583,160620230178,N11


In [ ]:
df_ent_all_select2 = df_ent_all_select[df_ent_all_select["entegration"] == "HepsiBurada"]
df_ent_all_select2.drop_duplicates(inplace=True)
df_ent_all_select2

C:\Users\user\AppData\Local\Temp\ipykernel_1528\1828841467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ent_all_select2.drop_duplicates(inplace=True)


,store_product_name,model,pov_productCode,entegration
8,"Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki...",003548,NaN,HepsiBurada
10,"Pikas Katlanabilir Masa, Kamp Masası, Plastik ...",002583,14421524554,HepsiBurada
14,Cool Direksiyon Katlanır Scooter Yükseklik Aya...,004173,NaN,HepsiBurada
27,Brilla Home Deri Taban Tekli Post Peluş Banyo ...,002643,371894852199,HepsiBurada
29,Cool Wheels Scooter Işıklı Pembe,004138,NaN,HepsiBurada
...,...,...,...,...
187604,Vipas Vipaş Wings Kalın Borulu Kanatlı Kurutm...,003263,NaN,HepsiBurada
187649,Okutan Hobi Montessori Küçük Çocuk Oyun Mutfağ...,Tanımsız.,NaN,HepsiBurada
187654,"Hobby life 3'lü Dikdöertgen Multi set (0,6+1,2...",001615,NaN,HepsiBurada
187817,Hyundai 21512-21010 Karter Tapası Kısa Unıversal,002792,NaN,HepsiBurada


In [ ]:
df_ent_all_select2.to_excel(r"C:\Users\user\Desktop\Hepsi Burada Son2.xlsx", index=False)

In [ ]:
df_ent_all_select3 = df_ent_all_select[df_ent_all_select["entegration"] == "N11"]
df_ent_all_select3.drop_duplicates(inplace=True)
df_ent_all_select3

C:\Users\user\AppData\Local\Temp\ipykernel_1528\921330967.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ent_all_select3.drop_duplicates(inplace=True)


,store_product_name,model,pov_productCode,entegration
49,Portatif Katlanabilir Kamp Piknik Masası 001 Gri,002583,220647971025,N11
98,Koruyucu Gümüş Göğüs Ucu Kapakları Large,003628,NaN,N11
100,Koruyucu Gümüş Göğüs Ucu Kapakları XLarge,003629,NaN,N11
194,Portatif Katlanabilir Kamp Piknik Masası 001 S...,002583,14421524554,N11
196,"Katlanabilir Bebek Yıkama Küvet Seti 4 Parça, ...",003943,NaN,N11
...,...,...,...,...
186298,"TERMOSTAT YUVASI LOGAN SANDERO CLIO SYMBOL 1,4...",Tanımsız.,NaN,N11
186613,Orijinal Çelik Zirkon Taşlı Su Yolu Bileklik K...,003654,NaN,N11
186828,Guard Bozuk Para Bölmeli Crazy Kahve Kadın Cüz...,Tanımsız.,NaN,N11
187620,Weili 93490-a4110 Direksiyon Zemberegi Cerato ...,002452,NaN,N11


In [ ]:
df_ent_all_select3.to_excel(r"C:\Users\user\Desktop\N11 Son2.xlsx", index=False)

In [ ]:
df_hepsi = pd.read_excel(r"C:\Users\user\Desktop\Hepsi Burada.xlsx")
df_hepsi.drop_duplicates(inplace=True)
df_hepsi

,store_product_name,SKU,Satıcı Stok Kodu
0,Cool Wheels Scooter Işıklı Pembe,HBCV000001X1UB,HBCV000001X1UB
1,Cool Wheels 12+ Orange Scooter,HBCV000003F3NY,HBCV000003F3NY
2,Awox Sparkling Kahve Makinesi Krom,HBCV00000AISQZ,HBCV00000AISQZ
3,"Hobby Life Alaska Nofrost Saklama Kabı 1,85 Li...",HBCV00001FDICE,ENT-HB604808211387
4,WB 2000 ml Büyük Kapasiteli Spor Su Şişesi (Yu...,HBCV00001MWJLC,HBCV00001MWJLC
...,...,...,...
907,Blue Focus Ingilizce Öğreniyorum BF111,HBV00000MMRYU,ENT-HB8681176321117
908,Merinos Fashion Micro Yorgan Zigzag Bej Çift K...,HBV00000P7WHE,HBV00000P7WHE
910,Awox Orion Tost Makinesi - Siyah,HBV00000Q8RY8,HBV00000Q8RY8
912,Hobby Life Özün Shop 15 Li Yumurta Saklama Kabı,HBV00000UT0BI,ENT-HB8694064019270


In [ ]:
df_hepsi["store_product_name"].nunique()

420

In [ ]:
df_hepsi["SKU"].nunique()

461

In [ ]:
df_hepsi["Satıcı Stok Kodu"].nunique()

461

In [ ]:
df_var = pd.merge(df_hepsi, df_ent_all_select, how="left", on="store_product_name")
df_var.drop_duplicates(inplace=True)
df_var

,store_product_name,SKU,Satıcı Stok Kodu,model,pov_productCode,entegration
0,Cool Wheels Scooter Işıklı Pembe,HBCV000001X1UB,HBCV000001X1UB,004138,NaN,HepsiBurada
1,Cool Wheels Scooter Işıklı Pembe,HBCV000001X1UB,HBCV000001X1UB,004138,NaN,Amazon
47,Cool Wheels Scooter Işıklı Pembe,HBCV000001X1UB,HBCV000001X1UB,Tanımsız.,NaN,HepsiBurada
100,Cool Wheels 12+ Orange Scooter,HBCV000003F3NY,HBCV000003F3NY,004164,NaN,HepsiBurada
103,Cool Wheels 12+ Orange Scooter,HBCV000003F3NY,HBCV000003F3NY,Tanımsız.,NaN,HepsiBurada
...,...,...,...,...,...,...
12912,Merinos Fashion Micro Yorgan Zigzag Bej Çift K...,HBV00000P7WHE,HBV00000P7WHE,Tanımsız.,NaN,HepsiBurada
12916,Awox Orion Tost Makinesi - Siyah,HBV00000Q8RY8,HBV00000Q8RY8,003722,NaN,HepsiBurada
13052,Awox Orion Tost Makinesi - Siyah,HBV00000Q8RY8,HBV00000Q8RY8,Tanımsız.,NaN,HepsiBurada
13064,Hobby Life Özün Shop 15 Li Yumurta Saklama Kabı,HBV00000UT0BI,ENT-HB8694064019270,001372,NaN,HepsiBurada


In [ ]:
df_var2 = df_var[df_var["entegration"] == "HepsiBurada"]
df_var2

,store_product_name,SKU,Satıcı Stok Kodu,model,pov_productCode,entegration
0,Cool Wheels Scooter Işıklı Pembe,HBCV000001X1UB,HBCV000001X1UB,004138,NaN,HepsiBurada
47,Cool Wheels Scooter Işıklı Pembe,HBCV000001X1UB,HBCV000001X1UB,Tanımsız.,NaN,HepsiBurada
100,Cool Wheels 12+ Orange Scooter,HBCV000003F3NY,HBCV000003F3NY,004164,NaN,HepsiBurada
103,Cool Wheels 12+ Orange Scooter,HBCV000003F3NY,HBCV000003F3NY,Tanımsız.,NaN,HepsiBurada
108,Awox Sparkling Kahve Makinesi Krom,HBCV00000AISQZ,HBCV00000AISQZ,003723,NaN,HepsiBurada
...,...,...,...,...,...,...
12912,Merinos Fashion Micro Yorgan Zigzag Bej Çift K...,HBV00000P7WHE,HBV00000P7WHE,Tanımsız.,NaN,HepsiBurada
12916,Awox Orion Tost Makinesi - Siyah,HBV00000Q8RY8,HBV00000Q8RY8,003722,NaN,HepsiBurada
13052,Awox Orion Tost Makinesi - Siyah,HBV00000Q8RY8,HBV00000Q8RY8,Tanımsız.,NaN,HepsiBurada
13064,Hobby Life Özün Shop 15 Li Yumurta Saklama Kabı,HBV00000UT0BI,ENT-HB8694064019270,001372,NaN,HepsiBurada


In [ ]:
df_var2 = df_var2.dropna(subset="pov_productCode")
df_var2.drop_duplicates(inplace=True)
df_var2

,store_product_name,SKU,Satıcı Stok Kodu,model,pov_productCode,entegration
400,Awox Kafija Kahve Makinesi - Rose Gold,HBCV00001X9ZS0,HBCV00001X9ZS0,004319,868407602029,HepsiBurada
531,Awox Kafija Kahve Makinesi - Rose Gold,HBCV00001X9ZS0,HBCV00001X9ZS0,003565,868407601526,HepsiBurada
944,Blooma Katlanır Plastik Sehpa Antrasıt-Kamp Ma...,HBCV00002MRKPW,HBCV00002MRKPW,002583,14421524554,HepsiBurada
1793,"Pikas Mocamp Devrilmez Ayaklı,taşınabilir Ocak...",HBCV00004B9T9Q,ENTV-HBAYK8683372407015CKMK,002263,AYK8683372407029ÇKMK,HepsiBurada
1800,"Pikas Mocamp Devrilmez Ayaklı,taşınabilir Ocak...",HBCV00004B9T9Q,ENTV-HBAYK8683372407015CKMK,002263,AYK8683372407015ÇKMK,HepsiBurada
...,...,...,...,...,...,...
12741,Brillahome Sierra-2 Kaymaz Taban Şönil Dokum...,HBCV0000594VJJ,ENTV-HB868407600906,003222,868407600910,HepsiBurada
12751,BrillaHome Sierra Kaymaz Taban Şönil Dokuma H...,HBCV00005BR5FB,ENTV-HB868407600931,003223,868407600944,HepsiBurada
12752,BrillaHome Sierra Kaymaz Taban Şönil Dokuma H...,HBCV00005BR5FB,ENTV-HB868407600931,003223,868407600950,HepsiBurada
12753,BrillaHome Sierra Kaymaz Taban Şönil Dokuma H...,HBCV00005BR5FB,ENTV-HB868407600931,003223,868407600938,HepsiBurada


In [ ]:
df_var2.to_excel(r"C:\Users\user\Desktop\Hepsi Burada Select.xlsx", index=False)

In [ ]:
df_hepsi_select = pd.read_excel(r"C:\Users\user\Desktop\Hepsi Burada Select.xlsx")
df_hepsi_select

,store_product_name,SKU,Satıcı Stok Kodu,model,pov_productCode,entegration
0,Awox Kafija Kahve Makinesi - Rose Gold,HBCV00001X9ZS0,HBCV00001X9ZS0,4319,868407602029,HepsiBurada
1,Awox Kafija Kahve Makinesi - Rose Gold,HBCV00001X9ZS0,HBCV00001X9ZS0,3565,868407601526,HepsiBurada
2,Blooma Katlanır Plastik Sehpa Antrasıt-Kamp Ma...,HBCV00002MRKPW,HBCV00002MRKPW,2583,14421524554,HepsiBurada
3,"Pikas Mocamp Devrilmez Ayaklı,taşınabilir Ocak...",HBCV00004B9T9Q,ENTV-HBAYK8683372407015CKMK,2263,AYK8683372407029ÇKMK,HepsiBurada
4,"Pikas Mocamp Devrilmez Ayaklı,taşınabilir Ocak...",HBCV00004B9T9Q,ENTV-HBAYK8683372407015CKMK,2263,AYK8683372407015ÇKMK,HepsiBurada
...,...,...,...,...,...,...
462,Brillahome Sierra-2 Kaymaz Taban Şönil Dokum...,HBCV0000594VJJ,ENTV-HB868407600906,3222,868407600910,HepsiBurada
463,BrillaHome Sierra Kaymaz Taban Şönil Dokuma H...,HBCV00005BR5FB,ENTV-HB868407600931,3223,868407600944,HepsiBurada
464,BrillaHome Sierra Kaymaz Taban Şönil Dokuma H...,HBCV00005BR5FB,ENTV-HB868407600931,3223,868407600950,HepsiBurada
465,BrillaHome Sierra Kaymaz Taban Şönil Dokuma H...,HBCV00005BR5FB,ENTV-HB868407600931,3223,868407600938,HepsiBurada


In [ ]:
df_hepsi_select["SKU"] = df_hepsi_select["SKU"].apply(lambda code: whitespace_del(code))
df_hepsi_select["Satıcı Stok Kodu"] = df_hepsi_select["Satıcı Stok Kodu"].apply(lambda code: whitespace_del(code))
df_hepsi_select

,store_product_name,SKU,Satıcı Stok Kodu,model,pov_productCode,entegration
0,Awox Kafija Kahve Makinesi - Rose Gold,HBCV00001X9ZS0,HBCV00001X9ZS0,4319,868407602029,HepsiBurada
1,Awox Kafija Kahve Makinesi - Rose Gold,HBCV00001X9ZS0,HBCV00001X9ZS0,3565,868407601526,HepsiBurada
2,Blooma Katlanır Plastik Sehpa Antrasıt-Kamp Ma...,HBCV00002MRKPW,HBCV00002MRKPW,2583,14421524554,HepsiBurada
3,"Pikas Mocamp Devrilmez Ayaklı,taşınabilir Ocak...",HBCV00004B9T9Q,ENTV-HBAYK8683372407015CKMK,2263,AYK8683372407029ÇKMK,HepsiBurada
4,"Pikas Mocamp Devrilmez Ayaklı,taşınabilir Ocak...",HBCV00004B9T9Q,ENTV-HBAYK8683372407015CKMK,2263,AYK8683372407015ÇKMK,HepsiBurada
...,...,...,...,...,...,...
462,Brillahome Sierra-2 Kaymaz Taban Şönil Dokum...,HBCV0000594VJJ,ENTV-HB868407600906,3222,868407600910,HepsiBurada
463,BrillaHome Sierra Kaymaz Taban Şönil Dokuma H...,HBCV00005BR5FB,ENTV-HB868407600931,3223,868407600944,HepsiBurada
464,BrillaHome Sierra Kaymaz Taban Şönil Dokuma H...,HBCV00005BR5FB,ENTV-HB868407600931,3223,868407600950,HepsiBurada
465,BrillaHome Sierra Kaymaz Taban Şönil Dokuma H...,HBCV00005BR5FB,ENTV-HB868407600931,3223,868407600938,HepsiBurada


In [ ]:
df_hepsi_select2 = df_hepsi_select[["SKU","Satıcı Stok Kodu","pov_productCode","model","entegration","store_product_name"]]
df_hepsi_select2

,SKU,Satıcı Stok Kodu,pov_productCode,model,entegration,store_product_name
0,HBCV00001X9ZS0,HBCV00001X9ZS0,868407602029,4319,HepsiBurada,Awox Kafija Kahve Makinesi - Rose Gold
1,HBCV00001X9ZS0,HBCV00001X9ZS0,868407601526,3565,HepsiBurada,Awox Kafija Kahve Makinesi - Rose Gold
2,HBCV00002MRKPW,HBCV00002MRKPW,14421524554,2583,HepsiBurada,Blooma Katlanır Plastik Sehpa Antrasıt-Kamp Ma...
3,HBCV00004B9T9Q,ENTV-HBAYK8683372407015CKMK,AYK8683372407029ÇKMK,2263,HepsiBurada,"Pikas Mocamp Devrilmez Ayaklı,taşınabilir Ocak..."
4,HBCV00004B9T9Q,ENTV-HBAYK8683372407015CKMK,AYK8683372407015ÇKMK,2263,HepsiBurada,"Pikas Mocamp Devrilmez Ayaklı,taşınabilir Ocak..."
...,...,...,...,...,...,...
462,HBCV0000594VJJ,ENTV-HB868407600906,868407600910,3222,HepsiBurada,Brillahome Sierra-2 Kaymaz Taban Şönil Dokum...
463,HBCV00005BR5FB,ENTV-HB868407600931,868407600944,3223,HepsiBurada,BrillaHome Sierra Kaymaz Taban Şönil Dokuma H...
464,HBCV00005BR5FB,ENTV-HB868407600931,868407600950,3223,HepsiBurada,BrillaHome Sierra Kaymaz Taban Şönil Dokuma H...
465,HBCV00005BR5FB,ENTV-HB868407600931,868407600938,3223,HepsiBurada,BrillaHome Sierra Kaymaz Taban Şönil Dokuma H...


In [ ]:
df_ham = pd.read_excel(r"C:\Users\user\Desktop\HB HAM.xlsx")
df_ham

,UniqueIdentifier,Buybox Sırası,Satıcı Stok Kodu,SKU
0,NaN,3,ENT-HB604808211318,HBCV00001FAQ4X
1,NaN,3,ENT-HB604808211271,HBCV00001FAQ2Y
2,NaN,1,ENT-HB604808211387,HBCV00001FDICE
3,NaN,1,ENT-HB604808211950,HBCV00001V218U
4,NaN,1,ENT-HB604808211967,HBCV00001V218N
...,...,...,...,...
1306,NaN,1,ENTV-HB47320017854,HBCV00006ZCBGQ
1307,NaN,1,ENT-HB8699243467674504-1,HBCV00006ZCCFG
1308,NaN,1,ENT-HB0037905486,HBCV00006ZCBUJ
1309,NaN,1,ENTV-HB32478610045,HBCV00006ZCEOT


In [ ]:
df_ham["SKU"] = df_ham["SKU"].apply(lambda code: whitespace_del(code))
df_ham

,UniqueIdentifier,Buybox Sırası,Satıcı Stok Kodu,SKU
0,NaN,3,ENT-HB604808211318,HBCV00001FAQ4X
1,NaN,3,ENT-HB604808211271,HBCV00001FAQ2Y
2,NaN,1,ENT-HB604808211387,HBCV00001FDICE
3,NaN,1,ENT-HB604808211950,HBCV00001V218U
4,NaN,1,ENT-HB604808211967,HBCV00001V218N
...,...,...,...,...
1306,NaN,1,ENTV-HB47320017854,HBCV00006ZCBGQ
1307,NaN,1,ENT-HB8699243467674504-1,HBCV00006ZCCFG
1308,NaN,1,ENT-HB0037905486,HBCV00006ZCBUJ
1309,NaN,1,ENTV-HB32478610045,HBCV00006ZCEOT


In [ ]:
df_ham_merge = pd.merge(df_ham, df_hepsi_select2, how="left", on=["Satıcı Stok Kodu","SKU"])
df_ham_merge.drop_duplicates(inplace=True)
df_ham_merge

,UniqueIdentifier,Buybox Sırası,Satıcı Stok Kodu,SKU,pov_productCode,model,entegration,store_product_name
0,NaN,3,ENT-HB604808211318,HBCV00001FAQ4X,NaN,NaN,NaN,NaN
1,NaN,3,ENT-HB604808211271,HBCV00001FAQ2Y,NaN,NaN,NaN,NaN
2,NaN,1,ENT-HB604808211387,HBCV00001FDICE,NaN,NaN,NaN,NaN
3,NaN,1,ENT-HB604808211950,HBCV00001V218U,NaN,NaN,NaN,NaN
4,NaN,1,ENT-HB604808211967,HBCV00001V218N,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1666,NaN,1,ENTV-HB47320017854,HBCV00006ZCBGQ,NaN,NaN,NaN,NaN
1667,NaN,1,ENT-HB8699243467674504-1,HBCV00006ZCCFG,NaN,NaN,NaN,NaN
1668,NaN,1,ENT-HB0037905486,HBCV00006ZCBUJ,NaN,NaN,NaN,NaN
1669,NaN,1,ENTV-HB32478610045,HBCV00006ZCEOT,NaN,NaN,NaN,NaN


In [ ]:
df_ham_merge.to_excel(r"C:\Users\user\Desktop\Hepsi Burada Ham.xlsx", index=False)

In [ ]:
df_hepsi_ham_edit = pd.read_excel(r"C:\Users\user\Desktop\Hepsi Burada Ham.xlsx")
df_hepsi_ham_edit

,UniqueIdentifier,Buybox Sırası,Satıcı Stok Kodu,SKU,pov_productCode
0,NaN,1,ENTV-HBAYK8683372407029CKMK,HBCV00004B9UVN,AYK8683372407029ÇKMK
1,NaN,1,ENTV-HBAYK8683372407029CKMK,HBCV00004B9UVN,AYK8683372407015ÇKMK
2,NaN,1,ENTV-HBAYK8683372407029CKMK,HBCV00004B9UVN,K8683372407016AÇKMK
3,NaN,1,ENTV-HBAYK8683372407015CKMK,HBCV00004B9T9Q,AYK8683372407029ÇKMK
4,NaN,1,ENTV-HBAYK8683372407015CKMK,HBCV00004B9T9Q,AYK8683372407015ÇKMK
...,...,...,...,...,...
429,NaN,1,ENT-HB00382145846,HBCV00005RMV7P,003854184287
430,NaN,1,ENT-HB003822456812,HBCV00005RMZE9,85412347447
431,NaN,1,ENT-HB0038254894,HBCV00005RNH3Y,96014840241
432,NaN,1,ENT-HB0038234854152,HBCV00005RNUZY,321017804546


In [ ]:
df_hepsi_ham_edit["SKU"].nunique()

74

In [ ]:
df_hepsi_ham_edit["Satıcı Stok Kodu"].nunique()

74

In [ ]:
df_ham

,UniqueIdentifier,Buybox Sırası,Satıcı Stok Kodu,SKU
0,NaN,3,ENT-HB604808211318,HBCV00001FAQ4X
1,NaN,3,ENT-HB604808211271,HBCV00001FAQ2Y
2,NaN,1,ENT-HB604808211387,HBCV00001FDICE
3,NaN,1,ENT-HB604808211950,HBCV00001V218U
4,NaN,1,ENT-HB604808211967,HBCV00001V218N
...,...,...,...,...
1306,NaN,1,ENTV-HB47320017854,HBCV00006ZCBGQ
1307,NaN,1,ENT-HB8699243467674504-1,HBCV00006ZCCFG
1308,NaN,1,ENT-HB0037905486,HBCV00006ZCBUJ
1309,NaN,1,ENTV-HB32478610045,HBCV00006ZCEOT


In [ ]:
df_ham["SKU"].nunique()

1310

In [ ]:
df_ham["Satıcı Stok Kodu"].nunique()

1311

#### Entegra All

In [73]:
df_prod_name_master = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Entegra\Product Name\Master\Entegra Model Product Name Update (11.09.2024).xlsx")
df_prod_name_master

,model,product_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
2,003954,Awox Barmix Çubuk Blender 1000 W
3,004286,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...
4,002656,Sprey Başlıklı Amber Cam 10 Ml Boş - Yedek Şişe
...,...,...
1258,003249,Merinos Relax Çift Kişilik Battaniye Color Li...
1259,004303,Procli̇s Deve Dikeni Yağı %100 Doğal Bitkisel ...
1260,003060,Merinos Eternity Fitilli Peluş Çift Kişilik Ne...
1261,003593,"Zirkon Taşlı Çelik Dünya Gezegen Gold Kolye, K..."


In [74]:
df_prod_name_master["model"] = df_prod_name_master["model"].apply(lambda x: whitespace_del(x))
df_prod_name_master["product_name"] = df_prod_name_master["product_name"].apply(lambda x: whitespace_del(x))
df_prod_name_master

,model,product_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
2,003954,Awox Barmix Çubuk Blender 1000 W
3,004286,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...
4,002656,Sprey Başlıklı Amber Cam 10 Ml Boş - Yedek Şişe
...,...,...
1258,003249,Merinos Relax Çift Kişilik Battaniye Color Li...
1259,004303,Procli̇s Deve Dikeni Yağı %100 Doğal Bitkisel ...
1260,003060,Merinos Eternity Fitilli Peluş Çift Kişilik Ne...
1261,003593,"Zirkon Taşlı Çelik Dünya Gezegen Gold Kolye, K..."


In [75]:
df_prod_name_master["model"].nunique()

1260

In [76]:
df_prod_name_master["product_name"].nunique()

1248

In [77]:
df_prod_name_master["product_name"].duplicated(keep=False).any()

True

In [78]:
df_prod_name_master[df_prod_name_master["product_name"].duplicated(keep=False)]

,model,product_name
27,002921,Baby Yuvarlak Torbalı Oyuncak/Kirli Çamaşır Se...
32,002940,Alas Çok Amaçlı Baza Kutusu Gri Çizgili Tekli ...
44,003706,"Ev18 Doğal Ağaç Çocuk Oyun Evi Demonte, Hediye..."
304,002375,"Ev18 Doğal Ağaç Çocuk Oyun Evi Demonte, Hediye..."
412,002920,Baby Yuvarlak Torbalı Oyuncak/Kirli Çamaşır Se...
502,002893,27 Parça Circle Toys Baby Puzzle Seti 12 Adet ...
535,004313,Şeffaf Tekerlekli Kedi Köpek Taşıma Valizi Çek...
587,002916,Brillahome Dijital Baskılı Pamuk Taban Yıkanab...
602,003683,Kadın Kalpli İnci Kolye Lüks Çelik Zincir Koly...
610,003676,Orijinal Çelik Zirkon Taşlı Su Yolu Kolye Kara...


In [79]:
df_enteg_all = pd.read_excel(r"C:\Users\user\Desktop\HB N11\Entegra Sales All.xlsx")
df_enteg_all

,id,order_number,datetime,entegration,status,status_name,company,cargo_code,invoice_country,invoice_city,...,cargo_fee_type_name,cargo_packet_type,cargo_packet_quantity,grand_total_desi,model,product_name,store_product_name,invoice_name,pov_productCode,total_product_quantity
0,188685,2317624676,2024-08-01 20:08:09,trendyol,2,Onaylandı,Ayberk Temel,7330016382584349,NaN,Kocaeli,...,Satıcı Öder,Dosya,1,0,002975,"%100 Doğal At Kılı Yüz Bakım Fırçası,Yüz Bakım...","%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...","%100 Doğal At Kılı Yüz Bakım Fırçası,Yüz Bakım...",NaN,2
1,142956,2176319833,2024-05-14 00:14:16,trendyol,2,Onaylandı,Beril Bahadır,7330014976704852,NaN,Ankara,...,Alıcı Öder,Dosya,1,0,002975,"%100 Doğal At Kılı Yüz Bakım Fırçası,Yüz Bakım...","%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...","%100 Doğal At Kılı Yüz Bakım Fırçası,Yüz Bakım...",NaN,2
2,139773,2167466542,2024-05-09 20:42:33,trendyol,2,Onaylandı,Aydın Caylan,7280014888930598,NaN,Sivas,...,Alıcı Öder,Dosya,1,0,002975,"%100 Doğal At Kılı Yüz Bakım Fırçası,Yüz Bakım...","%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...","%100 Doğal At Kılı Yüz Bakım Fırçası,Yüz Bakım...",NaN,2
3,134008,2149399085,2024-05-01 09:11:22,trendyol,2,Onaylandı,Songül KILINÇ,7330014705305541,NaN,İstanbul,...,Satıcı Öder,Dosya,1,0,002975,"%100 Doğal At Kılı Yüz Bakım Fırçası,Yüz Bakım...","%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...","%100 Doğal At Kılı Yüz Bakım Fırçası,Yüz Bakım...",NaN,3
4,113514,2065076170,2024-03-16 12:55:37,trendyol,2,Onaylandı,LAWEEN NANAKALI,7330013844234101,NaN,İstanbul,...,Alıcı Öder,Dosya,1,0,002975,"%100 Doğal At Kılı Yüz Bakım Fırçası,Yüz Bakım...","%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...","%100 Doğal At Kılı Yüz Bakım Fırçası,Yüz Bakım...",NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117222,83631,1935902477,2024-01-01 09:02:46,trendyol,2,Onaylandı,beyza İlhan,7280012517503588,NaN,Bursa,...,Alıcı Öder,Dosya,1,0,003548,"Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki...","Yüzük Zikirmatik Karışık Renk Pi9853, one size","Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki...",NaN,2
117223,83615,1935822681,2024-01-01 01:32:59,trendyol,2,Onaylandı,Şeyma Nur Altındağ,7280012517503809,NaN,Ankara,...,Alıcı Öder,Dosya,1,0,003548,"Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki...","Yüzük Zikirmatik Karışık Renk Pi9853, one size","Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki...",NaN,2
117224,83613,1935816283,2024-01-01 01:23:05,trendyol,2,Onaylandı,Şükran Dede,7280012517504011,NaN,Sakarya,...,Satıcı Öder,Dosya,1,0,003548,"Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki...","Yüzük Zikirmatik Karışık Renk Pi9853, one size","Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki...",NaN,3
117225,180405,203149362631-1,2024-07-14 19:25:00,N11,1,Yeni Siparis,Lefter çolakdemirci,112044694183347,NaN,Tekirdağ,...,N11 Öder,Dosya,1,0,003593,"Zirkon Taşlı Çelik Dünya Gezegen Gold Kolye, K...","Zirkon Taşlı Çelik Dünya Gezegen Gold Kolye, K...","Zirkon Taşlı Çelik Dünya Gezegen Gold Kolye, K...",NaN,1


In [80]:
df_enteg_all.columns

Index(['id', 'order_number', 'datetime', 'entegration', 'status',
       'status_name', 'company', 'cargo_code', 'invoice_country',
       'invoice_city', 'total', 'tax', 'grand_total',
       'store_order_status_name', 'cargo_company', 'cargo_fee_type_name',
       'cargo_packet_type', 'cargo_packet_quantity', 'grand_total_desi',
       'model', 'product_name', 'store_product_name', 'invoice_name',
       'pov_productCode', 'total_product_quantity'],
      dtype='object')

In [81]:
df_enteg_all_model_name = df_enteg_all[["model","product_name"]].drop_duplicates()
df_enteg_all_model_name.reset_index(drop=True, inplace=True)
# df_enteg_all_model_name.sort_values(by="product_name", inplace=True)
df_enteg_all_model_name

,model,product_name
0,002975,"%100 Doğal At Kılı Yüz Bakım Fırçası,Yüz Bakım..."
1,002932,0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim...
2,004113,1 Adet Ayakkabı Hurcu
3,004389,10 Adet Halı Kilim Koltuk Örtüsü Kaydırmaz Sab...
4,003470,"10 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sak..."
...,...,...
1459,003107,Yuvarlak Torbalı Oyuncak/kirli Çamaşır Sepeti ...
1460,003959,"Yuvarlak Yılan Zincir Bileklik, Pandora Charm ..."
1461,003548,"Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki..."
1462,003593,"Zirkon Taşlı Çelik Dünya Gezegen Gold Kolye, K..."


In [82]:
df_enteg_all_model_name["model"] = df_enteg_all_model_name["model"].apply(lambda x: whitespace_del(x))
df_enteg_all_model_name["product_name"] = df_enteg_all_model_name["product_name"].apply(lambda x: whitespace_del(x))
df_enteg_all_model_name

,model,product_name
0,002975,"%100 Doğal At Kılı Yüz Bakım Fırçası,Yüz Bakım..."
1,002932,0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim...
2,004113,1 Adet Ayakkabı Hurcu
3,004389,10 Adet Halı Kilim Koltuk Örtüsü Kaydırmaz Sab...
4,003470,"10 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sak..."
...,...,...
1459,003107,Yuvarlak Torbalı Oyuncak/kirli Çamaşır Sepeti ...
1460,003959,"Yuvarlak Yılan Zincir Bileklik, Pandora Charm ..."
1461,003548,"Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki..."
1462,003593,"Zirkon Taşlı Çelik Dünya Gezegen Gold Kolye, K..."


In [83]:
df_enteg_all_model_name["model"].nunique()

1127

In [84]:
df_enteg_all_model_name["product_name"].nunique()

1439

In [85]:
df_enteg_all_model_name.duplicated(keep=False).any()

False

In [86]:
df_enteg_all_model_name[df_enteg_all_model_name.duplicated(keep=False)]

,model,product_name


In [87]:
master_prod_name = set(df_prod_name_master["product_name"])
all_prod_name = set(df_enteg_all_model_name["product_name"])

In [88]:
master_prod_name.difference(all_prod_name)

{'0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim Kartları, Yavru Hayvanlar Circle Toys Dokun Hisset Crcl024',
 '1 Adet Ayakkabı Hurcu Gri Çizgili',
 '10 Metre Usb Kumandalı Peri Led 8 Animasyonlu Rbg Kg-3',
 '100 Adet Pratik Tazeliği Koruyan Tencere Ve Tabak Bonesi | Çok Amaçlı Kapak Gıda Bonesi',
 '100 Ledli Güneş Enerjili Bahçe Lambası Solar Hareket Sensörlü Şarj Dg1231',
 '10’Lu Geometrik Yumurta',
 '11 Parça Çelik Tencere Tava Seti, Çocuk Çeliktencere Tava Seti',
 '11 Parça Çelik Tencere Tava Seti, Çocuk Çeliktencere Tava Seti İçi Desenli',
 '1100 Ml Ölçekli Erzak Saklama Kabı',
 '114 Cm Kristal Havuz + 100 Adet 6cm Top + Pompa',
 '12li iğne gri',
 '2 Adet Nem Alıcı Aparat Rutubet Ve Küf Önleyici Kare Aparat Beyaz Naturel 2x400 gr',
 '2 Adet Paçuli Uçucu Yağı %100 Saf 10 ml ( Patchouli Oil )',
 '2 Adet Ylang Ylang Yağı 10 ml',
 '2 Litre Çift Kat Çelik Çay Termosu 1200A (Yeni̇ Model)',
 "2 No Berry Erzak Kabı (1,2 Lt.) 3'lü",
 '21512-21010 Karter Tapasi Kisa Universal',
 '2li Ayaklı

In [89]:
len(master_prod_name.difference(all_prod_name))

739

In [90]:
all_prod_name.difference(master_prod_name)

{'%100 Doğal Sarı Kantaron Yağı 50 ml',
 '%100 Doğal Çam Terebentin Yağı, Pine Turpentine Oil, 50 ml',
 '0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim Kartları, Yavru Hayvanlar circle toys dokun hisset CRCL024',
 "10 'lu Çanta Organizer SER039",
 '10 metre USB Kumandalı Peri Led 8 Animasyonlu RBG KG-3',
 '100 Adet Pratik Tazeliği Koruyan Tencere ve Tabak Bonesi | Çok Amaçlı Kapak Gıda Bonesi',
 '100 Ledli Güneş Enerjili Bahçe Lambası Solar Hareket Sensörlü Şarj DG1231',
 '11 Parça Çelik Tencere Tava Seti, Çocuk ÇelikTencere Tava Seti',
 '11 Parça Çelik Tencere Tava Seti, Çocuk ÇelikTencere Tava Seti İçi Desenli',
 '1100 ml Ölçekli Erzak Saklama Kabı',
 '2 Litre Çift Kat Çelik Çay Termosu 1200a (YENİ MODEL)',
 '21512-21010 KARTER TAPASI KISA UNIVERSAL',
 '3 BOGUMLU SUNSET HAVUZ 114x25cm',
 '3 Gözlü-bölmeli Çekmece Yeni Sezon Nano Teknoloji Kumaş 7620-LY1',
 '3 Gözlü-bölmeli Çekmece Yeni Sezon Nano Teknoloji Kumaş 7620-LY1, Tek Ebat',
 "3'lü Ahşap Boyama Seti",
 "3'lü Lazer Kesim Dekora

In [91]:
len(all_prod_name.difference(master_prod_name))

930

In [96]:
all_prod_diff_list = list(all_prod_name.difference(master_prod_name))
all_prod_diff_list

['Jüt Hasır Kapaklı Çamaşır sepeti,Kirli Sepeti,Oyuncak Sepeti ,Kapaklı Sepet,Banyo Sepeti XXL Krem',
 'PROCLİS Soğuk Sıkım %100 Doğal Kabak Çekirdeği Yağı, Pumpkin Seed Oil, 50 ml',
 'Alas Çok Amaçlı Mega Kutu Gri Çizgili Üçlü 60x40x30 Cm',
 'PROCLİS Deve Dikeni Yağı %100 Doğal Bitkisel Sabit Yağ, Milk Thistle Oil, Silybum Marianum 30 ml',
 'Soğuk Sıkım Hindistan Cevizi Yağı 337 ml Dil Sıyırıcı (TEMİZLEYİCİ)',
 'Intex351x145x48cm Seahawk 400 Bot Set, Kürek & Pompa 68351',
 'Aytuğ Kadın Modal Patik Çorap 5 Çift Özel Kutulu 4 Renk 33241',
 "6'lı Keyfi Ala Borosilikat Cam Kulplu Çay Bardağı 210ml Bardak",
 '0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim Kartları, Yavru Hayvanlar circle toys dokun hisset CRCL024',
 'Freeze Dry Çilek - Dondurularak Kurutulmuş Çilek - Meyve kurusu 15 gr',
 "Hobby Life 4 No Berry Erzak Kabı (2,4 Lt.) 2'li",
 'Natureva Çörekotu Yağı 10 ML',
 "4'lü Çok Amaçlı Dolap Içi Ve Çekmece Içi Düzenleyici Organizer Kutu Seti",
 'Müller MOTOR YAĞ KATKISI (ÜSTÜN ALMAN KAL

In [98]:
all_prod_name[all_prod_name["product_name"].isin(all_prod_diff_list)]

TypeError: 'set' object is not subscriptable

In [99]:
master_prod_model = set(df_prod_name_master["model"])
all_prod_model = set(df_enteg_all_model_name["model"])

In [100]:
all_prod_model.difference(master_prod_model)

{'001673PKTM',
 '003212',
 '004580',
 '004581',
 '813900291236',
 'ACRGRF00047',
 'TYC00541968775',
 'TYC00760581471',
 'TYC00847555459',
 'TYC00854424124',
 'TYCCV4VKIN170056964678741',
 'TYCDCFEF011795E680',
 'dop11822497igo',
 'muk-bgm6571',
 'wns-yıldızled'}

In [101]:
len(all_prod_model.difference(master_prod_model))

15

In [106]:
all_model_diff_list = list(all_prod_model.difference(master_prod_model))

In [107]:
all_prod_name[all_prod_name["model"].isin(all_model_diff_list)]

TypeError: 'set' object is not subscriptable